<a href="https://colab.research.google.com/github/Shubhammt/cs6910_assignment2/blob/main/CS6910_Assignment_2_Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet_v2 import ResNet50V2
from keras.models import Sequential
from keras.layers import Dense, Flatten

In [16]:
# Downloading the dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/wandb_datasets/nature_12K.zip \
    -O /home/data.zip

--2022-03-24 07:20:39--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘/home/data.zip’

/home/data.zip      100%[===================>]   3.55G   173MB/s    in 23s     

2022-03-24 07:21:01 (162 MB/s) - ‘/home/data.zip’ saved [3816687935/3816687935]



In [ ]:
import os
import zipfile
zip_path = '/home/data.zip'
#reading the zip file
zip_ref = zipfile.ZipFile(zip_path, 'r')
#extracting the zip file 
zip_ref.extractall('/home/dataset')
zip_ref.close()

In [18]:
train_dir = '/home/dataset/inaturalist_12K/train'
test_dir = '/home/dataset/inaturalist_12K/val'

In [3]:
wandb.login()

wandb: Currently logged in as: ma20m022 (use `wandb login --relogin` to force relogin)


True

In [4]:
wandb.init(project="Assignment 2", entity="shubham-argha")

wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [5]:
#train_dir = 'C:/Users/729sh/Desktop/inaturalist_12K/train'
#test_dir = 'C:/Users/729sh/Desktop/inaturalist_12K/val'

# Data Prep

In [6]:
# train, test, validation split
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, validation_split=0.1,subset="training",seed=123,image_size=(128, 128), batch_size=16
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, validation_split=0.1,subset="validation",seed=123,image_size=(128, 128), batch_size=16
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, image_size=(128, 128), batch_size=16
)

Found 9999 files belonging to 10 classes.
Using 9000 files for training.
Found 9999 files belonging to 10 classes.
Using 999 files for validation.
Found 2000 files belonging to 10 classes.


In [7]:
classes = train_ds.class_names
print(classes)

['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [8]:
for image_batch, labels_batch in train_ds:
    first_image = image_batch[0]
    print(np.max(first_image))
    print(image_batch.shape)
    print(labels_batch.shape)
    break

254.77734
(16, 128, 128, 3)
(16,)


In [9]:
# Normalization 
norm_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (norm_layer(x), y))
val_ds = val_ds.map(lambda x, y: (norm_layer(x), y))
test_ds = test_ds.map(lambda x, y: (norm_layer(x), y))

# Sweep and Experiments

In [10]:
# sweep configuration 
# method : random, bayes, grid
sweep_configuration = {
    'method': 'bayes', 
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'freeze_before' : {
            'values': [50, 60, 70]
        },
        'pretrained_models' :{
            'values' : ['InceptionV3','InceptionResNetV2', 'ResNet50V2', 'Xception']
        },     
        'FC_layer_size':{
            'values': [32, 64, 128, 256, 512]
        },
        'num_of_epochs':{
            'values': [5]
        }
    }
}

In [11]:
# sweep initialization
sweep_id = wandb.sweep(sweep_configuration, entity="shubham-argha", project="Assignment 2")

Create sweep with ID: pbgxgabk
Sweep URL: https://wandb.ai/shubham-argha/Assignment%202%20-Part%20B/sweeps/pbgxgabk


In [12]:
def train():
    
    default_configuration = {
        'freeze_before' : 50,
        'pretrained_models' : 'InceptionV3',
        'FC_layer_size': 32,
        'num_of_epochs' : 5
    }

    # wandb run initialization
    wandb.init(config=default_configuration)
    
    config = wandb.config

    input_shape = (128,128,3)

    if config.pretrained_models == 'InceptionV3':
        base_model =  InceptionV3(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = "model_IncV3_" + "FC_layer_size_" + str(config.FC_layer_size) 

    elif config.pretrained_models == 'InceptionResNetV2':
        base_model =  InceptionResNetV2(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = "model_IncResV2_" + "FC_layer_size_" + str(config.FC_layer_size)

    elif config.pretrained_models == 'ResNet50V2':
        base_model =  ResNet50V2(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = "model_Res50V2_" + "FC_layer_size_" + str(config.FC_layer_size)

    elif config.pretrained_models == 'Xception':
        base_model =  Xception(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = "model_Xcp_" + "FC_layer_size_" + str(config.FC_layer_size)


    #Code for fine tuning goes here
    base_model.trainable = True

    # Freeze layers
    freeze_point = len(base_model.layers) - config.freeze_before
    for layer in base_model.layers[:freeze_point]:
        layer.trainable = False
    
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(config.FC_layer_size, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy'])
    
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=config.num_of_epochs,
        callbacks = [WandbCallback()]
        )

In [ ]:
wandb.agent('o3tszjvl',train, count = 5)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: nuh0rcqo with config:
wandb: 	FC_layer_size: 64
wandb: 	freeze_before: 70
wandb: 	num_of_epochs: 5
wandb: 	pretrained_models: ResNet50V2
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/5
563/563 [==============================] - ETA: 0s - loss: 2.0442 - accuracy: 0.2822

C:\Users\729sh\Anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


563/563 [==============================] - 72s 107ms/step - loss: 2.0442 - accuracy: 0.2822 - val_loss: 2.8193 - val_accuracy: 0.3423
Epoch 2/5
563/563 [==============================] - 64s 114ms/step - loss: 1.7653 - accuracy: 0.3789 - val_loss: 2.2012 - val_accuracy: 0.2633
Epoch 3/5
563/563 [==============================] - 63s 111ms/step - loss: nan - accuracy: 0.2656 - val_loss: nan - val_accuracy: 0.0921
Epoch 4/5
563/563 [==============================] - 65s 115ms/step - loss: nan - accuracy: 0.1009 - val_loss: nan - val_accuracy: 0.0921
Epoch 5/5
563/563 [==============================] - 67s 118ms/step - loss: nan - accuracy: 0.1009 - val_loss: nan - val_accuracy: 0.0921


accuracy,▆█▅▁▁
epoch,▁▃▅▆█
loss,█▁
val_accuracy,█▆▁▁▁
val_loss,█▁
accuracy,0.10089
best_epoch,1
best_val_loss,2.20118
epoch,4
loss,nan
val_accuracy,0.09209


wandb: Agent Starting Run: 19h7ycgd with config:
wandb: 	FC_layer_size: 128
wandb: 	freeze_before: 50
wandb: 	num_of_epochs: 10
wandb: 	pretrained_models: ResNet50V2
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/10
563/563 [==============================] - 69s 115ms/step - loss: 1.9520 - accuracy: 0.3443 - val_loss: 2.1744 - val_accuracy: 0.4054
Epoch 2/10
563/563 [==============================] - 64s 113ms/step - loss: 1.5891 - accuracy: 0.4536 - val_loss: 1.8226 - val_accuracy: 0.4444
Epoch 3/10
563/563 [==============================] - 64s 113ms/step - loss: 1.3418 - accuracy: 0.5413 - val_loss: 1.7160 - val_accuracy: 0.4715
Epoch 4/10
563/563 [==============================] - 66s 116ms/step - loss: 1.0778 - accuracy: 0.6329 - val_loss: 2.0992 - val_accuracy: 0.4655
Epoch 5/10
563/563 [==============================] - 64s 114ms/step - loss: 0.8383 - accuracy: 0.7183 - val_loss: 2.0768 - val_accuracy: 0.4585
Epoch 6/10
563/563 [==============================] - 62s 110ms/step - loss: 0.8415 - accuracy: 0.7130 - val_loss: 43494.5117 - val_accuracy: 0.0921
Epoch 7/10
563/563 [==============================] - 66s 115ms/step - loss: 0.8905 - accuracy: 0.6961 - val_loss: 70.7353 - v

accuracy,▁▃▄▅▇▇▆█
epoch,▁▂▃▄▅▆▇█
loss,█▆▅▄▂▂▃▁
val_accuracy,▇████▁▁█
val_loss,▁▁▁▁▁█▁▁
accuracy,0.79667
best_epoch,2
best_val_loss,1.71597
epoch,7
loss,0.58853
val_accuracy,0.46446


wandb: ERROR Run 19h7ycgd errored: InternalError()
wandb: Agent Starting Run: 368aekmb with config:
wandb: 	FC_layer_size: 64
wandb: 	freeze_before: 70
wandb: 	num_of_epochs: 5
wandb: 	pretrained_models: ResNet50V2
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
